<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Анализ пользовательского взаимодействия с карточками статей 

Цели исследования: 
- Определить взаимодействия пользователей с карточками в системе с разбивкой по темам карточек;
- Определить как события генерируют источники с разными темами;
- Определить соотношения темы карточек и темы источников.

Данные исследования: составлены на основе приложения для просмотра и создания контента Яндекс.Дзен за 24.09.2019 c 18:28 до 19:00 из  источника данных: файл dash_visits.csv


Для менеджера по анализу контента был составлен дашборд, который будет использоваться не реже, чем раз в неделю.

Состав данных для дашборда:
- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;

По каким параметрам данные должны группироваться:
- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;

Характер данных:
- История событий по темам карточек — абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников — относительные величины (% событий);
- Соответствия тем источников темам карточек - абсолютные величины;

Дата-инженеры подготовили агрегирующую таблицу dash_visits, которая является источником данных для дашборда.

Вот её структура:
- record_id — первичный ключ,
- item_topic — тема карточки,
- source_topic — тема источника,
- age_segment — возрастной сегмент,
- dt — дата и время,
- visits — количество событий.


In [21]:
!pip install psycopg2_binary

In [126]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

#формируем текст запроса к базе SQL
query = ''' SELECT * 
        FROM dash_visits
    '''
#Сохраняем в переменную dash_visits данные, выгруженные
#из SQL в соответствии с запросом
dash_visits = pd.io.sql.read_sql(query, con = engine)

display(dash_visits.head())
display(dash_visits.info())

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


None

In [27]:
#выгружаем файл в формате csv для работы в Tableau
dash_visits.to_csv(r"dash_visits.csv", index=False)

In [28]:
dash_visits.sort_values(by=['item_topic','source_topic'])
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [113]:
# уникальные данные столбца item_topic
dash_visits['item_topic'].unique()

array(['Деньги', 'Дети', 'Женская психология', 'Женщины', 'Здоровье',
       'Знаменитости', 'Интересные факты', 'Искусство', 'История',
       'Красота', 'Культура', 'Наука', 'Общество', 'Отношения',
       'Подборки', 'Полезные советы', 'Психология', 'Путешествия',
       'Рассказы', 'Россия', 'Семья', 'Скандалы', 'Туризм', 'Шоу', 'Юмор'],
      dtype=object)

In [114]:
# уникальные данные столбца source_topic
dash_visits['source_topic'].unique()

array(['Авто', 'Деньги', 'Дети', 'Еда', 'Здоровье', 'Знаменитости',
       'Интерьеры', 'Искусство', 'История', 'Кино', 'Музыка', 'Одежда',
       'Полезные советы', 'Политика', 'Психология', 'Путешествия',
       'Ремонт', 'Россия', 'Сад и дача', 'Сделай сам',
       'Семейные отношения', 'Семья', 'Спорт', 'Строительство',
       'Технологии', 'Финансы'], dtype=object)

In [115]:
# уникальные данные столбца age_segment
dash_visits['age_segment'].unique()

array(['18-25', '26-30', '31-35', '36-40', '41-45', '45+'], dtype=object)

In [57]:
# формируем данные по столбцу item_topic
dash_visits_item_topic_total = dash_visits.groupby('item_topic')['visits'].sum()
dash_visits_item_topic_total

item_topic
Деньги                10291
Дети                  10989
Женская психология     7737
Женщины               11499
Здоровье              10399
Знаменитости           7394
Интересные факты      19942
Искусство              8516
История               15389
Красота                9814
Культура              10205
Наука                 21736
Общество              19640
Отношения             20666
Подборки              17772
Полезные советы       15435
Психология             8036
Путешествия            9260
Рассказы              10909
Россия                16966
Семья                 11897
Скандалы               9294
Туризм                 9512
Шоу                    7511
Юмор                   9398
Name: visits, dtype: int64

In [127]:
# формируем данные по столбцам item_topic и dt 
dash_visits_item_topic = dash_visits.groupby(['item_topic', 'dt'])['visits'].agg(sum).sort_values(ascending=False)
dash_visits_item_topic

item_topic          dt                 
Наука               2019-09-24 18:58:00    4372
                    2019-09-24 18:57:00    4277
Отношения           2019-09-24 18:58:00    4145
                    2019-09-24 18:57:00    4133
Интересные факты    2019-09-24 18:58:00    3910
                                           ... 
Искусство           2019-09-24 18:35:00       4
Знаменитости        2019-09-24 18:35:00       4
Туризм              2019-09-24 18:35:00       4
Женская психология  2019-09-24 18:35:00       3
Шоу                 2019-09-24 18:35:00       2
Name: visits, Length: 425, dtype: int64

In [128]:
# формируем данные по столбцам item_topic и dt(% от общего количества) 
dash_tota_item_topic_percent = (dash_visits_item_topic/dash_visits_item_topic_total*100).round(2)
dash_tota_item_topic_percent

item_topic          dt                 
Наука               2019-09-24 18:58:00    20.11
                    2019-09-24 18:57:00    19.68
Отношения           2019-09-24 18:58:00    20.06
                    2019-09-24 18:57:00    20.00
Интересные факты    2019-09-24 18:58:00    19.61
                                           ...  
Искусство           2019-09-24 18:35:00     0.05
Знаменитости        2019-09-24 18:35:00     0.05
Туризм              2019-09-24 18:35:00     0.04
Женская психология  2019-09-24 18:35:00     0.04
Шоу                 2019-09-24 18:35:00     0.03
Name: visits, Length: 425, dtype: float64

In [108]:
dash_visits_it = pd.pivot_table(dash_visits, index=['item_topic'], values=['visits'], aggfunc='sum')\
                .reset_index().sort_values(by='visits', ascending=False)
dash_visits_it

,item_topic,visits
11,Наука,21736
13,Отношения,20666
6,Интересные факты,19942
12,Общество,19640
14,Подборки,17772
19,Россия,16966
15,Полезные советы,15435
8,История,15389
20,Семья,11897
3,Женщины,11499


In [109]:
dash_visits_t

,item_topic,dt,visits
201,Наука,2019-09-24 18:58:00,4372
200,Наука,2019-09-24 18:57:00,4277
235,Отношения,2019-09-24 18:58:00,4145
234,Отношения,2019-09-24 18:57:00,4133
116,Интересные факты,2019-09-24 18:58:00,3910
...,...,...,...
126,Искусство,2019-09-24 18:35:00,4
92,Знаменитости,2019-09-24 18:35:00,4
381,Туризм,2019-09-24 18:35:00,4
41,Женская психология,2019-09-24 18:35:00,3


In [80]:
# формируем данные по столбцу source_topic 
dash_visits_c = pd.pivot_table(dash_visits, index=['source_topic'], values=['visits'], aggfunc='sum')\
                .reset_index().sort_values(by='visits', ascending=False)
# считаем % от общего количества
dash_visits_c['percent'] = (dash_visits_c['visits']/dash_visits['visits']\
                                  .sum()*100).round(2).sort_values(ascending=False)
dash_visits_c

,source_topic,visits,percent
20,Семейные отношения,33309,10.74
17,Россия,29831,9.62
12,Полезные советы,27412,8.84
15,Путешествия,24124,7.78
5,Знаменитости,23945,7.72
9,Кино,20084,6.47
2,Дети,15243,4.91
8,История,14628,4.72
21,Семья,13896,4.48
4,Здоровье,12831,4.14


Общие выводы:
Топ-5 событий по темам карточек:
 
- 1.Наука - 4372
- 2.Отношения - 4145
- 3.Интересные факты - 3910
- 4.Общество - 3897
- 5.Подборки – 3520

Топ-5 % событий по темам карточек :
- 1.История -9.184%
- 2.Полезные советы – 8.673%
- 3.Наука – 8.183%
- 4.Интересные факты – 7.375%
- 5.Подборки – 7.143%

Топ -5 событий по темам источника:
- 1.Семейные отношения – 10.74%
- 2.Россия – 9.62%
- 3.Полезные советы – 8.84%
- 4.Путешествия – 7.78%
- 5.Знаменитости – 7.72%

Топ-5 соответствий тем карточек темам источников:
- 1.Рассказы-путешествия - 4587
- 2.Общество-Россия – 3471
- 3.Наука-кино – 3279
- 4.Россия-Россия - 2847
- 5.Подборки-полезные советы – 2795



Презентация: <https://drive.google.com/file/d/1G_zCPwV2lVNlCsE61tv4pR9a270kaW6C/view?usp=sharing> 

Сcылка на дашборд: <https://public.tableau.com/app/profile/.36561146/viz/Book1_16863108352290/sheet4?publish=yes> 
